In [1]:
import asyncio
import json
import requests
from aiortc import RTCPeerConnection, RTCSessionDescription
import os


###############################################
# 1) Function to request an ephemeral API key #
###############################################
def get_ephemeral_token(standard_api_key: str) -> str:
    """
    Use your standard (secret) OpenAI API key to create an ephemeral key.
    The ephemeral key expires after ~1 minute, so request a new one each time.
    """
    url = "https://api.openai.com/v1/realtime/sessions"
    headers = {
        "Authorization": f"Bearer {standard_api_key}",
        "Content-Type": "application/json"
    }
    # The "model" below should match your desired Realtime model.
    # "voice" is optional; set it if you intend to stream/receive audio output.
    payload = {
        "model": "gpt-4o-realtime-preview-2024-12-17",
        "voice": "verse"
    }

    response = requests.post(url, headers=headers, json=payload)
    response.raise_for_status()  # Raise an error if the request failed
    data = response.json()

    # data should contain something like:
    # {
    #   "client_secret": {
    #       "value": "... ephemeral token ...",
    #       "expires_at": ...
    #   }
    # }
    ephemeral_key = data["client_secret"]["value"]
    return ephemeral_key


###############################################
# 2) Main async function to create the WebRTC #
#    connection to the Realtime API           #
###############################################
async def main():
    # -- Replace with your real standard API key here --
    STANDARD_API_KEY = os.getenv("OPENAI_API_KEY")
    
    # Get an ephemeral token for this session
    ephemeral_key = get_ephemeral_token(STANDARD_API_KEY)

    # Create our local peer connection
    pc = RTCPeerConnection()

    # Create the data channel for sending/receiving events
    channel = pc.createDataChannel("oai-events")

    @channel.on("message")
    def on_message(message: str):
        """
        Handle messages from the Realtime API.
        Each message is a JSON string representing a server-side event.
        """
        try:
            event = json.loads(message)
            print(">>> Received from Realtime API:", event)
        except Exception as e:
            print(">>> Error parsing message:", e)

    # Create a local SDP offer
    offer = await pc.createOffer()
    await pc.setLocalDescription(offer)

    # Send the SDP offer to OpenAI Realtime API
    base_url = "https://api.openai.com/v1/realtime"
    model_id = "gpt-4o-realtime-preview-2024-12-17"  # use whichever model you like

    sdp_headers = {
        "Authorization": f"Bearer {ephemeral_key}",
        "Content-Type": "application/sdp"
    }

    sdp_response = requests.post(
        f"{base_url}?model={model_id}",
        data=offer.sdp,
        headers=sdp_headers
    )

    # The response is the SDP "answer" from the server
    answer_sdp = sdp_response.text

    # Set that as the remote description on our RTCPeerConnection
    answer = RTCSessionDescription(sdp=answer_sdp, type="answer")
    await pc.setRemoteDescription(answer)

    print("WebRTC connection established with Realtime API.")
    print("You can now send messages labeled as `host1`, `host2`, or `user`.\n")
    
    # Simple loop to read input from user to mimic a 3-way conversation
    # Type lines like: host1: Hello from host1
    # or: user: Great to be here
    # or: host2: And I'm here too
    while True:
        line = input("Enter a message (e.g., `host1: Hello`), or just press Enter to quit:\n")
        line = line.strip()
        if not line:
            break
        
        # Simple parse: Expect the user to type "role: text"
        # e.g. "host1: Hello from me"
        if ":" in line:
            role, content = line.split(":", 1)
            role = role.strip()
            content = content.strip()
        else:
            # fallback
            role = "user"
            content = line

        # Construct a client event to send over the data channel
        # Adjust the event type/structure as needed by your use-case.
        # At minimum, you'll want to set the role/instructions or text for the model.
        event = {
            "type": "response.create",  # or "request", "response", etc. per your logic
            "response": {
                "modalities": ["text"],
                "role": role,
                "instructions": content
            }
        }

        channel.send(json.dumps(event))
        print(f"<<< Sent as {role}: {content}")

    print("Exiting... closing connection.")
    await pc.close()


###############################################
# 3) Entry point                              #
###############################################
if __name__ == "__main__":
    asyncio.run(main())


RuntimeError: This event loop is already running

In [5]:
from src.constants import RADIO_SYSTEM_PROMPT, SONG_DIALOGUE_PROMPT, NEWS_SUMMARY_PROMPT
from src.spotify_handler import SpotifyHandler

spotify_handler = SpotifyHandler(username="leo.camacho1738")

## Make the search feature search for exclusively playlists with +1 songs on it
spotify_handler.search_for_playlist("J Cole")





{'playlists': {'href': 'https://api.spotify.com/v1/search?offset=0&limit=1&query=J%20Cole%20best%20songs&type=playlist',
  'limit': 1,
  'next': 'https://api.spotify.com/v1/search?offset=1&limit=1&query=J%20Cole%20best%20songs&type=playlist',
  'offset': 0,
  'previous': None,
  'total': 900,
  'items': [{'collaborative': False,
    'description': '',
    'external_urls': {'spotify': 'https://open.spotify.com/playlist/7w6OtQzHfsXUNloBCJE1FI'},
    'href': 'https://api.spotify.com/v1/playlists/7w6OtQzHfsXUNloBCJE1FI',
    'id': '7w6OtQzHfsXUNloBCJE1FI',
    'images': [{'height': None,
      'url': 'https://image-cdn-ak.spotifycdn.com/image/ab67706c0000da848f1a8f379e83c00846759603',
      'width': None}],
    'name': "J Cole's BEST Songs",
    'owner': {'display_name': 'Kung Fu Kenny',
     'external_urls': {'spotify': 'https://open.spotify.com/user/shayne.b654321'},
     'href': 'https://api.spotify.com/v1/users/shayne.b654321',
     'id': 'shayne.b654321',
     'type': 'user',
     'ur